In [ ]:
import numpy as np
import pandas as pd
from sympy import *
import random

In [ ]:

import csv
data2=[]
file_path = 'input_unbounded_4d.csv'
with open(file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)

    # Iterate through each row in the CSV file
    for row in csv_reader:
        # Each row is a list of values
        data2.append(row)

In [ ]:
#Converting char data to int
data=[]
for i in data2:
    l=[]
    for j in i:
        l.append(float(j))
    data.append(l)

z=data[0][0:-1]
c=data[1][0:-1]
data1=data[2:]

a=[]
b=[]
for i in data1:
    a.append(i[:-1])
    b.append(i[-1])

m=len(a)
n=len(a[0])
a=np.array(a)
b=np.array(b)
z=np.array(z)


In [ ]:
z=np.array([26,0,0])

In [ ]:
b

array([  0.,  10.,  10., -25.])

In [ ]:
a

array([[-1., -1., -1.],
       [ 0.,  1.,  1.],
       [ 0.,  0., -1.],
       [-1.,  0., -1.]])

In [ ]:
#Checking the feasility
def point_in_region(a,b,x):
    return a @ x <= b

def step1(a,b,z):
    check=point_in_region(a,b,z)
    for i in check:
        if not i:
            return False
    return True

In [ ]:
step1(a,b,z)

True

In [ ]:
#counting the tight rows

def no_of_tight(a,b,x):
    tight_count=0
    for i in range(len(a)):
        if round(a[i] @ x.T,4) ==round(b[i],4):
            tight_count+=1
    return tight_count

In [ ]:
#checking whether we are at a vertex

def check_vertex(a1):
    if len(a1)==0:
        return False
    if np.linalg.matrix_rank(a1)==n:
        return True
    else:
        return False

In [ ]:
#split the matrix based on tight rows and untight rows

def spliting_matrix(a,b,x):
    a1=[]
    a2=[]
    b1=[]
    b2=[]
    #print(a)
    for i in range(a.shape[0]):
        # print("%%%%%%%%%%%%%%%%%")
        # print(a[i])
        # print(b[i])
        # print(a[i]@x.T)
        # print("#############")
        if round(a[i] @ x.T,4) ==round(b[i],4):
            a1.append(a[i])
            b1.append(b[i])
        else:
            a2.append(a[i])
            b2.append(b[i])
    a1=np.array(a1)
    a2=np.array(a2)
    b1=np.array(b1)
    b2=np.array(b2)
    return a1,a2,b1,b2

In [ ]:
def replace_zero(ar):
  eps = 0.0000000000000001
  count = 0
  for i in ar:
    if(i == 0):
      i = eps
      ar[count] = i
    count+=1
  return ar

In [ ]:
#reaching a vertex
#Step 2
def step2(a,b,x):
    tight_rows=no_of_tight(a,b,x)
    a1,a2,b1,b2=spliting_matrix(a,b,x)

    while not check_vertex(a1):

        if len(a1)==0:
            a1=np.array([]).reshape((0, n))

        u = np.array(Matrix(a1).nullspace()).astype(float)
        u = u.flatten()
        u = np.reshape(u, (-1, n))
        u_hat = u / np.linalg.norm(u,axis=1,keepdims=True)
        u_hat=u_hat[0]

        dino = (np.dot(a2 , u_hat))

        dino = replace_zero(dino)

        alpha  = (b2 -np.dot(a2,x.T) )/ dino.T
        # print("alpha",alpha)
        alpha = [i for i in alpha if abs(i)<= 1e+15 and abs(i)!=0]
        alpha_1 = []
        # removing -ve alphas
        for aa in alpha:
            if aa > 0:
                alpha_1.append(aa)

        # print("alpha_1",alpha_1)
        # taking min of alphas
        if len(alpha_1) == 0:
          alpha_1 = max(alpha)
        else:
          alpha_1 = min(alpha_1)
        # print("alpha_1_1",alpha_1)
        u_hat = u_hat.T
        x = x + alpha_1 * u_hat
        # print("x",x)
        a1,a2,b1,b2=spliting_matrix(a,b,x)
    return x

In [ ]:
step2(a,b,z)

array([-10., -25.,  35.])

In [ ]:
#checking whether current point is an optimum
def check_optimum(a1,c):
    # print(a1)
    alphas=np.linalg.solve(a1.T, c)
    # print(alphas)
    for i in alphas:
        if i<0:
            return False
    return True

In [ ]:
#Function to get optimum point
#Step 3
def step3(a,b,x,c):

    a1,a2,b1,b2=spliting_matrix(a,b,x)
    # print("------------------------------------------")
    # print("a1",a1)
    # print("hello")
    # print("c",c)
    while not check_optimum(a1,c):
        # print("a1",a1)
        # print(a1.shape)
        # print(c.shape)
        print("****************************************")
        print("Present Vertex point= ",x)
        a1i=np.linalg.inv(a1).T
        print(a1)
        alphas=np.linalg.solve(a1.T, c)
          # print(alphas)
          # print(len(alphas))
        u=[]
        for i in range(len(alphas)):
            if alphas[i] < 0:
                u=a1i[i]
                break
        u=np.array(u)
        u=-1*u
        u=u.T
        dino = np.dot(a2 , u)

        dino = replace_zero(dino)

        alpha  = (b2 -np.dot(a2,x.T) )/ dino
          # print("--------------")
          # print("prev alpha",alpha)
          #alpha = alpha[0]
        alpha_1 = []
          # removing -ve alphas
        cnt = 0
        for aa in alpha:
            if aa > 0:
                cnt+=1
                alpha_1.append(aa)

        if(len(alpha_1) == 0):
          print("Unbounded result Optimum cannot be found !!")
          return []

          # taking min of alphas
        alpha_1 = min(alpha_1)
        u_hat = u.T
          # print("alpha_1",alpha_1)
          # print("prev x",x)
        x = x + alpha_1 * u_hat

          # print("x=",x)
        a1,a2,b1,b2=spliting_matrix(a,b,x)
        # print("------------")
        # print("a1",a1)
        # print("------------")
    return x

In [ ]:
if step1(a,b,z):
  res = step3(a,b,step2(a,b,z),c)
  print("Optimum point",res)
else:
  print("Given point is not feasible")

[[-1. -1. -1.]
 [ 0.  1.  1.]
 [-1.  0. -1.]]
****************************************
Present Vertex point=  [-10. -25.  35.]
[[-1. -1. -1.]
 [ 0.  1.  1.]
 [-1.  0. -1.]]
Unbounded result Optimum cannot be found !!
Optimum point []
